# SGM Baseline

*Throughout, much of the code is adaped from the Argoverse stereo tutorial.*

## 1 Setup

----

Ensure you're in <...>/argoverse-api

In [1]:
import os
os.chdir('..')
print(f"Current directory: {os.getcwd()}")

Current directory: /Users/main/Documents/GitHub/argoverse-api


In [16]:
import cv2
import numpy as np

from tqdm import tqdm
from pathlib import Path
from argoverse.data_loading.stereo_dataloader import ArgoverseStereoDataLoader
from argoverse.evaluation.stereo.eval import StereoEvaluator
from argoverse.utils.calibration import get_calibration_config
from argoverse.utils.camera_stats import RECTIFIED_STEREO_CAMERA_LIST

STEREO_FRONT_LEFT_RECT = RECTIFIED_STEREO_CAMERA_LIST[0]
STEREO_FRONT_RIGHT_RECT = RECTIFIED_STEREO_CAMERA_LIST[1]

In [10]:
main_dir = "/Users/main/Documents/GitHub/argoverse-api/"
data_dir = f"{main_dir}argoverse-stereo_v1.1/"

## 2 Main Model

----

Goal: Predict disparity map from pair of stereo images

### 2.1 Training

Notes: 

 * SGM is not a learning algorithm, so we don't need to do any training.  As such, here we just define the model
for use in the testing section below.

In [4]:
# Defining the SGM parameters (please check the OpenCV documentation for details).
# We found this parameters empirically and based on the Argoverse Stereo data. 
max_disp = 192
win_size = 10
uniqueness_ratio = 15
speckle_window_size = 200
speckle_range = 2
block_size = 11
P1 = 8 * 3 * win_size ** 2
P2 = 32 * 3 * win_size ** 2

# Defining the Weighted Least Squares (WLS) filter parameters.
lmbda = 0.1
sigma = 1.0

# Defining the SGM left matcher.
left_matcher = cv2.StereoSGBM_create(
    minDisparity=0,
    numDisparities=max_disp,
    blockSize=block_size,
    P1=P1,
    P2=P2,
    disp12MaxDiff=max_disp,
    uniquenessRatio=uniqueness_ratio,
    speckleWindowSize=speckle_window_size,
    speckleRange=speckle_range,
    mode=cv2.STEREO_SGBM_MODE_SGBM_3WAY,
)

# Defining the SGM right matcher needed for the left-right consistency check in the WLS filter.
right_matcher = cv2.ximgproc.createRightMatcher(left_matcher)

# Defining the WLS filter.
wls_filter = cv2.ximgproc.createDisparityWLSFilter(matcher_left=left_matcher)
wls_filter.setLambda(lmbda)
wls_filter.setSigmaColor(sigma)

### 2.2 Evaluation

In [21]:
stereo_data_loader = ArgoverseStereoDataLoader(data_dir, "val")

metrics = []
log_ids = os.listdir(f"{data_dir}rectified_stereo_images_v1.1/val/")
i = 0
for log_id in log_ids:
    i += 1
    print(f"Now evaluating log_id :: \t\t {i}/{len(log_ids)}")
    left_stereo_img_fpaths = stereo_data_loader.get_ordered_log_stereo_image_fpaths(
        log_id=log_id, 
        camera_name=STEREO_FRONT_LEFT_RECT)
    right_stereo_img_fpaths = stereo_data_loader.get_ordered_log_stereo_image_fpaths(
        log_id=log_id, 
        camera_name=STEREO_FRONT_RIGHT_RECT)
    disparity_map_fpaths = stereo_data_loader.get_ordered_log_disparity_map_fpaths(
        log_id=log_id,
        disparity_name="stereo_front_left_rect_disparity")
    disparity_obj_map_fpaths = stereo_data_loader.get_ordered_log_disparity_map_fpaths(
        log_id=log_id,
        disparity_name="stereo_front_left_rect_objects_disparity")

    for idx in tqdm(range(len(left_stereo_img_fpaths))):
        # Load the testing image and corresponding disparity and foreground disparity maps
        stereo_front_left_rect_image = stereo_data_loader.get_rectified_stereo_image(left_stereo_img_fpaths[idx])
        stereo_front_right_rect_image = stereo_data_loader.get_rectified_stereo_image(right_stereo_img_fpaths[idx])
        stereo_front_left_rect_disparity = stereo_data_loader.get_disparity_map(disparity_map_fpaths[idx])
        stereo_front_left_rect_objects_disparity = stereo_data_loader.get_disparity_map(disparity_obj_map_fpaths[idx])

        # Apply the model
        left_disparity = left_matcher.compute(stereo_front_left_rect_image, stereo_front_right_rect_image)
        right_disparity = right_matcher.compute(stereo_front_right_rect_image, stereo_front_left_rect_image)
        left_disparity_pred = wls_filter.filter(left_disparity, stereo_front_left_rect_image, None, right_disparity)
        left_disparity_pred = np.float32(left_disparity_pred) / 16.0
        left_disparity_pred[left_disparity_pred < 0] = 0

        left_disparity_pred = np.uint16(left_disparity_pred * 256.0)
        timestamp = int(Path(disparity_map_fpaths[idx]).stem.split("_")[-1])
        save_dir_disp = f"{main_dir}707-files/results/sgm/stereo_output/{log_id}/"
        Path(save_dir_disp).mkdir(parents=True, exist_ok=True)
        filename = f"{save_dir_disp}/disparity_{timestamp}.png"
        cv2.imwrite(filename, left_disparity_pred)

    pred_dir = Path(save_dir_disp)
    gt_dir = Path(f"{data_dir}/disparity_maps_v1.1/val/{log_id}")
    save_figures_dir = Path(f"/tmp/results/sgm/figures/{log_id}/")
    save_figures_dir.mkdir(parents=True, exist_ok=True)

    evaluator = StereoEvaluator(
        pred_dir,
        gt_dir,
        save_figures_dir,
    )
    metrics += [evaluator.evaluate()]

Now evaluating log_id :: 		 1/17


100%|██████████| 74/74 [08:02<00:00,  6.52s/it]


Now evaluating log_id :: 		 2/17


100%|██████████| 73/73 [07:31<00:00,  6.18s/it]


Now evaluating log_id :: 		 3/17


...

Now evaluating log_id :: 		 16/17


100%|██████████| 74/74 [07:09<00:00,  5.81s/it]


Now evaluating log_id :: 		 17/17


100%|██████████| 68/68 [06:30<00:00,  5.74s/it]


{'all:10': 0.7100484384214201, 'fg:10': 1.558911822412393, 'bg:10': 0.5167476920782379, 'all*:10': 0.37796714014820926, 'fg*:10': 1.0862981992874448, 'bg*:10': 0.22049208589777788, 'all:5': 1.1174777709430592, 'fg:5': 1.8753392551496433, 'bg:5': 0.9448997174132555, 'all*:5': 0.6840675011899369, 'fg*:5': 1.3839757813118867, 'bg*:5': 0.5284649860094401, 'all:3': 1.3167011646824633, 'fg:3': 2.0210125106899786, 'bg:3': 1.1563173957405914, 'all*:3': 0.8588047798708927, 'fg*:3': 1.5408426312188002, 'bg*:3': 0.7071751904916316}


{
    'all:10': 0.7100484384214201, 
    'fg:10': 1.558911822412393, 
    'bg:10': 0.5167476920782379, 
    'all*:10': 0.37796714014820926, 
    'fg*:10': 1.0862981992874448, 
    'bg*:10': 0.22049208589777788, 
    'all:5': 1.1174777709430592, 
    'fg:5': 1.8753392551496433, 
    'bg:5': 0.9448997174132555, 
    'all*:5': 0.6840675011899369, 
    'fg*:5': 1.3839757813118867, 
    'bg*:5': 0.5284649860094401, 
    'all:3': 1.3167011646824633, 
    'fg:3': 2.0210125106899786, 
    'bg:3': 1.1563173957405914, 
    'all*:3': 0.8588047798708927, 
    'fg*:3': 1.5408426312188002, 
    'bg*:3': 0.7071751904916316
}

In [28]:
# Assuming images already generated:

stereo_data_loader = ArgoverseStereoDataLoader(data_dir, "val")

metrics = []
lens = []
log_ids = [
    'f9fa3960-537f-3151-a1a3-37a9c0d6d7f7',
    '1d676737-4110-3f7e-bec0-0c90f74c248f',
    'da734d26-8229-383f-b685-8086e58d1e05',
    '6db21fda-80cd-3f85-b4a7-0aadeb14724d',
    '85bc130b-97ae-37fb-a129-4fc07c80cca7',
    '33737504-3373-3373-3373-633738571776',
    '033669d3-3d6b-3d3d-bd93-7985d86653ea',
    'f1008c18-e76e-3c24-adcc-da9858fac145',
    '5ab2697b-6e3e-3454-a36a-aba2c6f27818',
    'cb762bb1-7ce1-3ba5-b53d-13c159b532c8',
    '70d2aea5-dbeb-333d-b21e-76a7f2f1ba1c',
    '2d12da1d-5238-3870-bfbc-b281d5e8c1a1',
    '64724064-6472-6472-6472-764725145600',
    '00c561b9-2057-358d-82c6-5b06d76cebcf',
    'cb0cba51-dfaf-34e9-a0c2-d931404c3dd8',
    'e9a96218-365b-3ecd-a800-ed2c4c306c78',
    '39556000-3955-3955-3955-039557148672'
]
for log_id in tqdm(log_ids):
    left_stereo_img_fpaths = stereo_data_loader.get_ordered_log_stereo_image_fpaths(
        log_id=log_id, 
        camera_name=STEREO_FRONT_LEFT_RECT)
    right_stereo_img_fpaths = stereo_data_loader.get_ordered_log_stereo_image_fpaths(
        log_id=log_id, 
        camera_name=STEREO_FRONT_RIGHT_RECT)
    disparity_map_fpaths = stereo_data_loader.get_ordered_log_disparity_map_fpaths(
        log_id=log_id,
        disparity_name="stereo_front_left_rect_disparity")
    disparity_obj_map_fpaths = stereo_data_loader.get_ordered_log_disparity_map_fpaths(
        log_id=log_id,
        disparity_name="stereo_front_left_rect_objects_disparity")
    lens += [len(left_stereo_img_fpaths)]

    # save_dir_disp = f"{main_dir}707-files/results/sgm/stereo_output/{log_id}/"
    # pred_dir = Path(save_dir_disp)
    # gt_dir = Path(f"{data_dir}/disparity_maps_v1.1/val/{log_id}")
    # save_figures_dir = Path(f"/tmp/results/sgm/figures/{log_id}/")
    # save_figures_dir.mkdir(parents=True, exist_ok=True)

    # evaluator = StereoEvaluator(
    #     pred_dir,
    #     gt_dir,
    #     save_figures_dir,
    # )
    # metrics += [evaluator.evaluate()]

100%|██████████| 17/17 [06:48<00:00, 24.03s/it]


In [29]:
metrics

[{'all:10': 3.4682411890098845,
  'fg:10': 3.2607305562718696,
  'bg:10': 3.4987609043347843,
  'all*:10': 2.127099557781579,
  'fg*:10': 1.4296237075896268,
  'bg*:10': 2.2315293677764965,
  'all:5': 6.040719663877293,
  'fg:5': 4.845705995442148,
  'bg:5': 6.21647681178234,
  'all*:5': 3.805137290672996,
  'fg*:5': 2.1154658067066108,
  'bg*:5': 4.058123930964382,
  'all:3': 7.088952627235509,
  'fg:3': 5.337551548391019,
  'bg:3': 7.346540692197659,
  'all*:3': 4.414083257319883,
  'fg*:3': 2.4891531848764408,
  'bg*:3': 4.70229407015799},
 {'all:10': 5.268346514373905,
  'fg:10': 11.613081046814493,
  'bg:10': 4.383715782483582,
  'all*:10': 2.6297636772578956,
  'fg*:10': 6.30691097033541,
  'bg*:10': 2.1737669383930975,
  'all:5': 10.43713117510306,
  'fg:5': 17.92786957540344,
  'bg:5': 9.392715942349787,
  'all*:5': 6.508537396648399,
  'fg*:5': 9.170360533352476,
  'bg*:5': 6.178449267718411,
  'all:3': 13.084019479317138,
  'fg:3': 21.17194713559252,
  'bg:3': 11.956339617304